In [1]:
from langchain_groq import ChatGroq
from langchain_community.document_loaders import UnstructuredURLLoader
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
urls = ['https://www.victoriaonmove.com.au/local-removalists.html','https://www.victoriaonmove.com.au/index.html','https://www.victoriaonmove.com.au/contact.html']
loader = UnstructuredURLLoader(urls=urls)
data = loader.load()

In [3]:
data

[Document(metadata={'source': 'https://www.victoriaonmove.com.au/local-removalists.html'}, page_content='Loading...\n\nLOCAL REMOVALS\n\nYour trusted partner in seamless moving and packing solutions!\n\nGoogle logo G\n\nGoogle Rating\n\nGold star icon\n\nGold star icon\n\nGold star icon\n\nGold star icon\n\nGold star icon\n\n5 stars, 111 reviews\n\n\n\nRequst A call for You:\n\nLocal removal services via "Victoria on move"\n\nVictoria on Move is your trusted local moving company in Melbourne, specializing in seamless relocation services. As experienced furniture movers and relocation experts, we provide top-notch packing and moving services tailored to your needs. Whether you\'re moving across town or relocating interstate, our professional movers ensure a stress-free experience. Count on Victoria on Move for reliable removal services, making us the preferred choice among local movers in Melbourne. Discover why we\'re recognized for our commitment to quality and customer satisfaction.\

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
docs = text_splitter.split_documents(data)

print(docs)

[Document(metadata={'source': 'https://www.victoriaonmove.com.au/local-removalists.html'}, page_content='Loading...\n\nLOCAL REMOVALS\n\nYour trusted partner in seamless moving and packing solutions!\n\nGoogle logo G\n\nGoogle Rating\n\nGold star icon\n\nGold star icon\n\nGold star icon\n\nGold star icon\n\nGold star icon\n\n5 stars, 111 reviews\n\n\n\nRequst A call for You:\n\nLocal removal services via "Victoria on move"\n\nVictoria on Move is your trusted local moving company in Melbourne, specializing in seamless relocation services. As experienced furniture movers and relocation experts, we provide top-notch packing and moving services tailored to your needs. Whether you\'re moving across town or relocating interstate, our professional movers ensure a stress-free experience. Count on Victoria on Move for reliable removal services, making us the preferred choice among local movers in Melbourne. Discover why we\'re recognized for our commitment to quality and customer satisfaction.\

In [5]:
len(docs)

11

In [6]:
docs[2]

Document(metadata={'source': 'https://www.victoriaonmove.com.au/local-removalists.html'}, page_content='Icon\n\nPacking and Unpacking Services\n\nOptional packing and unpacking services available to save you time and effort, using high-quality packing materials.\n\nIcon\n\nCustomized Moving Plans\n\nTailored moving plans to fit your specific requirements and schedule, providing personalized service and peace of mind.\n\nIcon\n\nProfessional Team\n\nExperienced and courteous moving professionals dedicated to delivering exceptional service and customer satisfaction throughout the entire moving process.\n\nFind removalists travelling to or from Melbourne\n\nFrom Melbourne\n\nTo Melbourne\n\nMoving from Ballarat to MelbourneMoving from Bendigo to MelbourneMoving from Geelong to MelbourneMoving from Melton to MelbourneMoving from Mildura to MelbourneMoving from Mooroopna to MelbourneMoving from Shepparton to MelbourneMoving from Sunbury to MelbourneMoving from Traralgon to MelbourneMoving f

In [7]:
from langchain_huggingface import HuggingFaceEmbeddings

In [8]:
embedding = HuggingFaceEmbeddings(model_name = "ibm-granite/granite-embedding-278m-multilingual")

d:\learning\practice_python\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\learning\practice_python\venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Vikas Chauhan\.cache\huggingface\hub\models--ibm-granite--granite-embedding-278m-multilingual. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrato

KeyboardInterrupt: 

In [ ]:
embedding.embed_query("Delhi is the capital of India")

In [ ]:
doc_embedding = embedding.embed_documents(docs)

In [ ]:
from langchain_chroma import Chroma

In [ ]:
vectorstore = Chroma.from_documents(documents=docs, embedding=embedding)

In [ ]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k":6})

retrieved_docs = retrievevr.invoke("What kind of services they provide?")

In [ ]:
llm = ChatGroq(model_name="Llama3-8b-8192", temperature=0)

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [ ]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [ ]:
response = rag_chain.invoke({"input": "What kind of services they provide?"})
print(response["answer"])